In [100]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
%matplotlib inline

In [101]:
df = pd.read_csv('exercise_conversion_data.csv')

In [102]:
df.head()

,user_id,bike_model_id,city,created_date,subscription_date
0,110,1.0,Emeryville,5/1/20,NaN
1,111,4.0,Alameda,5/13/20,NaN
2,112,5.0,Berkeley,5/1/20,NaN
3,113,NaN,Oakland,5/2/20,NaN
4,114,5.0,Emeryville,5/4/20,NaN


In [103]:
df['subscribed'] = ~df.subscription_date.isna()

In [104]:
df.created_date = pd.to_datetime(df.created_date)
df.subscription_date = pd.to_datetime(df.subscription_date)

In [105]:
df[df.subscribed==True]

,user_id,bike_model_id,city,created_date,subscription_date,subscribed
6,116,NaN,Berkeley,2020-05-08,2020-05-15,True
7,117,7.0,Berkeley,2020-05-06,2020-05-13,True
8,118,7.0,Oakland,2020-04-17,2020-04-24,True
9,119,1.0,San Leandro,2020-05-12,2020-05-19,True
10,120,7.0,Oakland,2020-04-30,2020-05-07,True
...,...,...,...,...,...,...
415,529,7.0,Alameda,2020-05-11,2020-05-18,True
416,530,1.0,Berkeley,2020-04-27,2020-05-04,True
419,533,8.0,Oakland,2020-05-06,2020-05-13,True
427,541,8.0,Alameda,2020-04-22,2020-04-29,True


In [106]:
df.created_date.dt.month_name()

0        May
1        May
2        May
3        May
4        May
       ...  
442      NaN
443    April
444    April
445      May
446      May
Name: created_date, Length: 447, dtype: object

In [107]:
df['has_concierge'] = df.created_date.dt.month_name().isin(['May'])

In [108]:
df_1 = df[['bike_model_id','city','subscribed','has_concierge']]

In [109]:
df_1.shape

(447, 4)

#### Encode the target variable, subscribed, to binary

In [110]:
df_1.subscribed = df_1.subscribed.astype('uint8')
df_1.has_concierge = df_1.has_concierge.astype('uint8')

/Users/johnsylvester1/anaconda2/envs/newvenv/lib/python3.6/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [111]:
df_2 = pd.get_dummies(df_1, columns=['city','bike_model_id'], prefix=None)

In [112]:
df_2

,subscribed,has_concierge,city_Alameda,city_Berkeley,city_Emeryville,city_Oakland,city_San Leandro,bike_model_id_1.0,bike_model_id_4.0,bike_model_id_5.0,bike_model_id_7.0,bike_model_id_8.0
0,0,1,0,0,1,0,0,1,0,0,0,0
1,0,1,1,0,0,0,0,0,1,0,0,0
2,0,1,0,1,0,0,0,0,0,1,0,0
3,0,1,0,0,0,1,0,0,0,0,0,0
4,0,1,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
442,0,0,0,0,0,0,0,0,0,1,0,0
443,0,0,1,0,0,0,0,0,1,0,0,0
444,0,0,0,0,0,0,1,0,0,0,0,1
445,0,1,0,0,0,1,0,0,0,1,0,0


### Data Exploration

In [113]:
df.subscribed.value_counts(normalize=True)

False    0.749441
True     0.250559
Name: subscribed, dtype: float64

In [114]:
df_1.groupby(by='subscribed').mean()

,bike_model_id,has_concierge
subscribed,,
0,4.949833,0.641791
1,5.280374,0.687500


In [115]:
x=df_2.iloc[:,1:]
y=df_2['subscribed']

In [116]:
x

,has_concierge,city_Alameda,city_Berkeley,city_Emeryville,city_Oakland,city_San Leandro,bike_model_id_1.0,bike_model_id_4.0,bike_model_id_5.0,bike_model_id_7.0,bike_model_id_8.0
0,1,0,0,1,0,0,1,0,0,0,0
1,1,1,0,0,0,0,0,1,0,0,0
2,1,0,1,0,0,0,0,0,1,0,0
3,1,0,0,0,1,0,0,0,0,0,0
4,1,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
442,0,0,0,0,0,0,0,0,1,0,0
443,0,1,0,0,0,0,0,1,0,0,0
444,0,0,0,0,0,1,0,0,0,0,1
445,1,0,0,0,1,0,0,0,1,0,0


In [118]:
import statsmodels.api as sm

logit_model=sm.Logit(y,x)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: inf
         Iterations 7
                         Results: Logit
Model:                Logit             Pseudo R-squared:  inf   
Dependent Variable:   subscribed        AIC:               inf   
Date:                 2020-11-11 18:17  BIC:               inf   
No. Observations:     447               Log-Likelihood:    -inf  
Df Model:             10                LL-Null:           0.0000
Df Residuals:         436               LLR p-value:       1.0000
Converged:            1.0000            Scale:             1.0000
No. Iterations:       7.0000                                     
-----------------------------------------------------------------
                   Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
-----------------------------------------------------------------
has_concierge      0.6476   0.2624  2.4676 0.0136  0.1332  1.1619
city_Alameda      -1.1518   0.4561 -2.5254 0.0116 -2.0458 -0.2579
city_

/Users/johnsylvester1/anaconda2/envs/newvenv/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1747: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/Users/johnsylvester1/anaconda2/envs/newvenv/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1800: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))
/Users/johnsylvester1/anaconda2/envs/newvenv/lib/python3.6/site-packages/statsmodels/base/model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/Users/johnsylvester1/anaconda2/envs/newvenv/lib/python3.6/site-packages/statsmodels/base/model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


### Remove insignificant columns

In [122]:
df_2.columns.to_list()

['subscribed',
 'has_concierge',
 'city_Alameda',
 'city_Berkeley',
 'city_Emeryville',
 'city_Oakland',
 'city_San Leandro',
 'bike_model_id_1.0',
 'bike_model_id_4.0',
 'bike_model_id_5.0',
 'bike_model_id_7.0',
 'bike_model_id_8.0']

In [124]:
cols = [
 'has_concierge',
 'city_Alameda',
 'city_Berkeley',
 'city_Oakland',
 'city_San Leandro',
 'bike_model_id_4.0',
 'bike_model_id_5.0']

In [125]:
x=df_2[cols]
y=df_2['subscribed']

### Need to drop NaNs

### Rerun the model with the shortened list of columns

In [126]:

logit_model=sm.Logit(y,x)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: inf
         Iterations 7
                         Results: Logit
Model:                Logit             Pseudo R-squared:  inf   
Dependent Variable:   subscribed        AIC:               inf   
Date:                 2020-11-11 18:22  BIC:               inf   
No. Observations:     447               Log-Likelihood:    -inf  
Df Model:             6                 LL-Null:           0.0000
Df Residuals:         440               LLR p-value:       1.0000
Converged:            1.0000            Scale:             1.0000
No. Iterations:       7.0000                                     
-----------------------------------------------------------------
                   Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
-----------------------------------------------------------------
has_concierge      0.5148   0.2373  2.1693 0.0301  0.0497  0.9799
city_Alameda      -0.8536   0.3538 -2.4123 0.0159 -1.5471 -0.1601
city_

/Users/johnsylvester1/anaconda2/envs/newvenv/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1747: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/Users/johnsylvester1/anaconda2/envs/newvenv/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1800: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))
/Users/johnsylvester1/anaconda2/envs/newvenv/lib/python3.6/site-packages/statsmodels/base/model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/Users/johnsylvester1/anaconda2/envs/newvenv/lib/python3.6/site-packages/statsmodels/base/model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
